### C# Interactive Notebook - Carbon Example No.3

This example notebook demonstrates how a .NET client app can use the `Carbon.Examples.WebService.Common` NuGet package to make strongly-typed calls to the [Carbon web service](https://rcsapps.azurewebsites.net/carbon/swagger/). Although this notebook acts as a .NET client, REST style web services are designed to be platform neutral and can work with all modern languages that have REST service support, such as Java, JavaScript, Rust, Kotlin, Swift, and many more.

Referencing the **Carbon.Examples.WebService.Common** NuGet package will download and cache all the required libraries, which are then named in using statements for coding convenience.

In [1]:
#r "nuget: Carbon.Examples.WebService.Common"
using Carbon.Examples.WebService.Common;
using RCS.Carbon.Shared;

Installed Packages Carbon.Examples.WebService.Common, 1.0.0

Create an instance of the service client and make a `GetServiceInfo` call to sanity check that the service is responding correctly. Some interesting values in the response are displayed.

In [2]:
var client = new CarbonServiceClient("https://rcsapps.azurewebsites.net/carbon/", "Example03/1.0");
ServiceInfo info = await client.GetServiceInfo();
Console.WriteLine($"Host Machine ....... {info.HostMachine}");
Console.WriteLine($"Host Account ....... {info.HostAccount}");
Console.WriteLine($"Service Version .... {info.Version}");
Console.WriteLine($"Service Build ...... {info.Build}");
Console.WriteLine($"Carbon Version ..... {info.CarbonVersion}");
Console.WriteLine($"Carbon Build ....... {info.CarbonBuild}");

Host Machine ....... RD0003FF85C098
Host Account ....... rcsapps
Service Version .... 1.0.0.0
Service Build ...... 2022-11-02 12:17 GMT+11
Carbon Version ..... 8.3.20.0
Carbon Build ....... 2022-10-29 13:34 GMT+11


The call to `StartSessionFree` starts a web service session using the *free* account that is provided by Red Centre Software for testing and demonstrations. The response contains a *Session Id* which is held inside the client to indicate that subsequent calls are authorised. The response also contains interesting information, such as a hierarchical set of customers, jobs and variable trees that the account is authorised to access. Apps with a GUI could present the session information in tree controls or other selector controls, but in this case the Ids and names are simply displayed.

In [3]:
SessionInfo sess = await client.StartSessionFree("Example Notebook");
Console.WriteLine($"Session Id ........ {sess.SessionId}");
Console.WriteLine($"Account Id ........ {sess.Id}");
Console.WriteLine($"Account Name ...... {sess.Name}");
Console.WriteLine($"Entity Name ....... {sess.EntityName}");
foreach (var cust in sess.SessionCusts)
{
    Console.WriteLine($"|  {cust.Id} {cust.Name}");
    foreach (var job in cust.SessionJobs)
    {
        string jobline = $"{job.Id} {job.Name}";
        if (job.VartreeNames?.Length > 0)
        {
            jobline += " [" + string.Join(",", job.VartreeNames) + "]";
        }
        Console.WriteLine($"|  |  {jobline}");
    }
}

Session Id ........ Z38KY8QSGL
Account Id ........ 10000010
Account Name ...... free
Entity Name ....... KNRCustomers
|  3625340368 rcsruby
|  |  25303847 demo [vartree,TsapiTree]
|  |  21372990 minidemo
|  3382340368 rcspublic
|  |  29938684 gridwatchuk [VarTree]
|  |  23464980 uk-ge2019 [VarTree]
|  |  27811105 cdc-covid [VarTree]
|  |  27468214 firstfleet [VarTree]
|  |  25895325 stockmarkets [VarTree]
|  |  24098249 gss [VarTree]
|  |  23325371 aemo [VarTree]
|  |  22826258 romeo-juliet [vartree]
|  |  24821945 cov19-global [VarTree]


The `OpenCloudJob` call opens one of the public demo *jobs* published by Red Centre Software. A *job* is a complete set of related data, variables and settings that generate cross-tabulation reports, must like a folder is used to group together related files. Only one job can be open and associated with the client session at any time.

A default specification properties object is created. A display specification object is created and one of its properties is adjusted to indicate that TSV (tab separated value) output format is required. The two objects can be used to customise the behaviour and appearance of corss-tabulation reports in great detail, but for this example only one property needs to be changed.

The `GenTab` call generates the report using the variables "Age" and "Region" as the top and side variables respectively. The report is a set of text lines that are displayed.

In [4]:
OpenCloudJobResponse openresp = await client.OpenCloudJob("rcsruby", "demo", tocType: JobTocType.None);
var sprops = new XSpecProperties();
var dprops = new XDisplayProperties();
dprops.Output.Format = XOutputFormat.TSV;
string[] report = await client.GenTab(null, "Age", "Region", null, null, sprops, dprops);
foreach (string line in report)
{
    Console.WriteLine($"| {line}");
}

| Name: GenTab2
| Top: Age
| Side: Region
| 	15-25	26-35	36-50	51-65	65+	Under 35	Over 35	Under 50	Over 50
| NE	41	53	44	45	55	94.00	144.00	138.00	100.00
| SE	43	48	49	52	46	91.00	147.00	140.00	98.00
| SW	48	38	42	67	52	86.00	161.00	128.00	119.00
| NW	40	58	68	58	53	98.00	179.00	166.00	111.00
| East	84	101	93	97	101	185.00	291.00	278.00	198.00
| West	88	96	110	125	105	184.00	340.00	294.00	230.00
| North	81	111	112	103	108	192.00	323.00	304.00	211.00
| South	91	86	91	119	98	177.00	308.00	268.00	217.00


:star: **Discussion point** &mdash; Cross-tabulation reports can be generated in a variety of formats such as TSV, CSV, HTML, XML, JSON and XLSX (an Excel workbook). Apps can choose the most suitable format for display on their runtime platform.

When the calling app is finished with the client it should issue a `CloseJob` call to release any resources associated with a previously opened job, then `Dispose` of the client class instance.

In [5]:
await client.CloseJob();
client.Dispose();